In [ ]:
import os
from glob import glob
import wfdb
import numpy as np
import pandas as pd
import scipy
from tqdm import tqdm

In [ ]:
root = "/Users/tanmaygoyal/Desktop/Assignments and Events/Biomedicine/ECG_Project/data/PTB_XL"

In [ ]:
hr_records = []

original_data_source = "/Users/tanmaygoyal/Desktop/Assignments and Events/Biomedicine/Datasets/PTB_XL_data"
with open(original_data_source + "/RECORDS" , 'r') as f:
    records = f.readlines()
for r in records:   
    if "records500" in r:
        hr_records.append(r.strip("\n"))

for r in hr_records:
    data , m = wfdb.rdsamp(original_data_source + "/" + r)
    dict = {}
    dict['val'] = data
    file_name = r.split("/")[-1].split("_")[0] + ".mat"
    scipy.io.savemat(file_name = file_name , mdict = dict)

In [ ]:
data_records = []

with open(os.path.join(root, "RECORDS"), "r") as f:
    for line in f:
        if "records500" in line:
            data_records.append(os.path.join(root , line.strip()))

In [ ]:
meta_info = pd.read_csv("/Users/tanmaygoyal/Desktop/Assignments and Events/Biomedicine/ECG_Project/data/PTB_XL/ptbxl_database.csv")
meta_info['scp_codes']


In [ ]:
scp_codes_df = pd.read_csv("/Users/tanmaygoyal/Desktop/Assignments and Events/Biomedicine/ECG_project/data/PTB_XL/scp_statements.csv")
scp_codes_df.rename(columns = {'Unnamed: 0' : 'Potential_diagnosis'} , inplace = True)

In [ ]:
print(meta_info.columns)
new_df = meta_info[['filename_hr' , 'ecg_id' , 'age' , 'sex' , 'scp_codes']]

In [ ]:
# finding the non-zero values in scp_codes_column
collection_non_zero_strings = []
for row in new_df.iterrows():
    non_zero_string = ""
    dict_string = row[1]['scp_codes']
    dict_string_split = dict_string.split(",")
    dict_string_split = [x.split(":") for x in dict_string_split]
    for x in dict_string_split:
        x[0] = x[0].strip(' ')
        x[0] = x[0].strip("{")
        x[0] = x[0].strip(' ')
        x[0] = x[0].strip('\'')
        x[1] = x[1].strip(' ')
        x[1] = x[1].strip('}')
        x[1] = float(x[1])
        if x[1]!=0:
            non_zero_string = non_zero_string + x[0] + " "
    collection_non_zero_strings.append(non_zero_string)

new_df['potential_diagnosis'] = collection_non_zero_strings
new_df.drop(columns = ['scp_codes'] , inplace = True)
split_diagnosis = new_df['potential_diagnosis'].str.split(" " , expand = True)
new_df = pd.concat((new_df , split_diagnosis) , axis = 1)
new_df.drop(columns = ['potential_diagnosis'] , inplace = True)
new_df.rename(columns = {0 : "diagnosis_0" , 1 : "diagnosis_1" , 2 : "diagnosis_2" , 3 : "diagnosis_3" , 4 : "diagnosis_4" , 5 : "diagnosis_5"} , inplace = True)

In [ ]:
new_df_other_columns = [c for c in new_df.columns if c != "filename_hr"]
new_df['patient_id'] = new_df['filename_hr'].apply(lambda x: x.split("/")[-1].split("_")[0])
new_df_new_columns = ['patient_id'] + new_df_other_columns
new_df = new_df[new_df_new_columns]
new_df.to_csv("/Users/tanmaygoyal/Desktop/Assignments and Events/Biomedicine/ECG_Project/data/PTB_XL/reference.csv" , index = False , header = True)

In [ ]:
# creating a table for the diagnosis vs number of values
columns = ['diagnosis_' + str(i) for i in range(6)]
unique_diagnosis = new_df['diagnosis_0'].unique().tolist()
for col in columns[1:]:
    unique_diagnosis += new_df[col].unique().tolist()
unique_diagnosis = list(set(unique_diagnosis))
unique_diagnosis.remove('')
dict_counts = {}
for diagnosis in unique_diagnosis:
    dict_counts[diagnosis] = []
    for col in columns:
        try:
            dict_counts[diagnosis].append(new_df[col].value_counts()[diagnosis])
        except:
            dict_counts[diagnosis].append(0)

count_df = pd.DataFrame(dict_counts).T
count_df.rename(columns = {i : "diagnosis_" + str(i) for i in range(6)} , inplace = True)
for col in count_df.columns:
    count_df.loc[None , col] =  len(new_df) - count_df[col].sum()
count_df

In [ ]:
for col in count_df.columns:
    print(col , count_df[col].sum())

In [ ]:
df = pd.read_csv("/Users/tanmaygoyal/Desktop/Assignments and Events/Biomedicine/ECG_Project/data/CPSC/labels.csv")

In [ ]:
df.columns

In [ ]:
label_csv_path ="/Users/tanmaygoyal/Desktop/Assignments and Events/Biomedicine/ECG_Project/data/PTB_XL/label.csv"

In [ ]:
new_df

In [ ]:
# collecting all potential labels
label_csv = new_df['filename_hr'].to_frame()

potential_labels = []
for cols in ['diagnosis_'+ str(i) for i in range(6)]:
    potential_labels += new_df[cols].tolist()
    potential_labels = list(set(potential_labels))
potential_labels.remove(None)
potential_labels.remove('')

for label in (potential_labels):
    label_csv[label] = 0
    for col in ['diagnosis_'+ str(i) for i in range(6)]:
        all_with_label1 = new_df[new_df[col] == label]['filename_hr'].tolist()      
        label_csv.loc[label_csv['filename_hr'].isin(all_with_label1) , label] = 1

label_csv['patient_id'] = label_csv['filename_hr'].apply(lambda x: x.split("/")[-1].split("_")[0])
label_csv.drop(columns = ["filename_hr"] , axis = 1 , inplace = True)
label_csv_columns = ['patient_id']+potential_labels
label_csv = label_csv[label_csv_columns]

In [ ]:
label_csv.to_csv(label_csv_path , index = False , header = True)

In [ ]:
label_csv[potential_labels].sum()